In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("job_postings.csv")

In [3]:
df.head()

,job_id,company_id,title,description,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,...,expiry,closed_time,formatted_experience_level,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type
0,85008768,NaN,Licensed Insurance Agent,While many industries were hurt by the last fe...,52000.0,NaN,45760.0,YEARLY,Full-time,"Chico, CA",...,1.710000e+12,NaN,NaN,NaN,1.690000e+12,NaN,1,FULL_TIME,USD,BASE_SALARY
1,133114754,77766802.0,Sales Manager,Are you a dynamic and creative marketing profe...,NaN,NaN,NaN,NaN,Full-time,"Santa Clarita, CA",...,1.700000e+12,NaN,NaN,NaN,1.690000e+12,NaN,0,FULL_TIME,NaN,NaN
2,133196985,1089558.0,Model Risk Auditor,Join Us as a Model Risk Auditor – Showcase You...,NaN,NaN,NaN,NaN,Contract,"New York, NY",...,1.700000e+12,NaN,NaN,NaN,1.690000e+12,NaN,0,CONTRACT,NaN,NaN
3,381055942,96654609.0,Business Manager,Business ManagerFirst Baptist Church ForneyFor...,NaN,NaN,NaN,NaN,Full-time,"Forney, TX",...,1.700000e+12,NaN,NaN,NaN,1.690000e+12,NaN,0,FULL_TIME,NaN,NaN
4,529257371,1244539.0,NY Studio Assistant,YOU COULD BE ONE OF THE MAGIC MAKERS\nKen Fulk...,NaN,NaN,NaN,NaN,Full-time,"New York, NY",...,1.710000e+12,NaN,NaN,NaN,1.690000e+12,NaN,1,FULL_TIME,NaN,NaN


In [ ]:
df.shape

# Data Processing

## Stop Words Removal

In [ ]:
from nltk.corpus import stopwords
# import nltk
# nltk.download('stopwords')
stop_words = stopwords.words('english')
stop_words.append('nan')
stop_words.append('removed')
stop_words.remove('not')
stop_words.remove('no')
stop_words[:10]

In [ ]:
df['Data'] = df['Data'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [ ]:
df.head()

## Tokenise Words & Put into Matrix

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

In [ ]:
# Create a CountVectorizer instance
vectorizer = CountVectorizer()

# Preparing the data to fit_tranformation
listofData  = list(df["Data"].array)

# Create the Matrix
matrix = vectorizer.fit_transform(listofData)
matrix = matrix.toarray()

# Add into new column in df
df["Vector"] = [row.tolist() for row in matrix]
df

## Calculating L2 Distance

In [ ]:
def l2_distance(v1, v2):
    if (len(v1) != len(v2)):
        print("Vectors not of the same dimension")
        return -1
    sum = 0
    for i, pt in enumerate(v1):
        sum += (pt - v2[i])**2
    return sum ** 0.5

In [ ]:
l2_distance(df['Vector'][0], df['Vector'][1])

In [ ]:
l2DistanceDf = df.copy()
v1 = df['Vector'][0]
l2DistanceDf["Distance from v1"] = df['Vector'].apply(lambda x: l2_distance(v1,x))
l2DistanceDf

## Calculating Cosine Similarity

In [ ]:
def cosine_sim(v1,v2):
    if (len(v1) != len(v2)):
        print("Vectors not of the same dimension")
        return -1
    dotProduct = 0
    sumOfSquaredV1 = 0
    sumOfSquaredV2 = 0
    for i, pt in enumerate(v1):
        dotProduct += pt*v2[i]
        sumOfSquaredV1 += pt*pt
        sumOfSquaredV2 += v2[i] * v2[i]
    
    cosineSimilarity = dotProduct/((sumOfSquaredV1 ** 0.5) * (sumOfSquaredV2 ** 0.5))
    return cosineSimilarity

In [ ]:
print(cosine_sim(df['Vector'][0], df['Vector'][1]))
print(cosine_sim(df['Vector'][0], df['Vector'][0]))

In [ ]:
cosSimDf = df.copy()
v1 = df['Vector'][0]
cosSimDf["Cosine Similarity with v1"] = df['Vector'].apply(lambda x: cosine_sim(v1,x))
cosSimDf

In [ ]:
cosSimDf[cosSimDf["Cosine Similarity with v1"] != 0]

In [ ]:
print(cosSimDf["Data"][0])
print(cosSimDf["Data"][119])
print(cosSimDf["Data"][len(cosSimDf['Data'])-1])

In [ ]:
import time
# sample_queries = ["Looking for newest Fragrant Dior perfume",
#                   "Hottest Gucci trends",
#                   "Polo ralph lauren for fathers day",
#                   "Balenciaga latest products",
#                   "Prada leather handbags",
#                   "Rolex submariner on sale",
#                   "Louis Vuitton black bag",
#                   "Tiffany And Co Bracelet",
#                   "Chanel pink bag",
#                   "London gucci shoes"]

sample_queries = ["beautiful bags", "horrible watch bad design", "Bad Rolex"]
elapse_time = []

for query in sample_queries:
    tempDf = df[['Data']].copy()
    start_time = time.time()
    
    new_record = pd.DataFrame([{'Data':query}])
    tempDf = pd.concat([tempDf, new_record], ignore_index=True)
    
    # Create a CountVectorizer instance
    vectorizer = CountVectorizer()
    # Preparing the data to fit_tranformation
    listofData  = list(tempDf["Data"].array)
    # Create the Matrix
    matrix = vectorizer.fit_transform(listofData)
    matrix = matrix.toarray()
    # Creating a Vector column column in df
    tempDf["Vector"] = [row.tolist() for row in matrix]
    
    print(tempDf['Data'][len(tempDf['Data'])-1])
    v1 = tempDf['Vector'][len(tempDf['Data'])-1]
    tempDf['Vector'] = tempDf['Vector'].apply(lambda x: cosine_sim(v1,x))
    
    end_time = time.time()
    
    tempDf.to_csv("Linear Search Results//"+query.replace(" ","_")+".csv")
    
    elapse_time.append(end_time-start_time)

print(elapse_time)

In [ ]:
tempDf[tempDf['Vector'] > 0.5]

In [ ]:
[10.759878635406494, 10.649078607559204, 10.477964639663696, 10.413733005523682, 10.690922498703003, 11.268934488296509, 10.590667247772217, 10.398170709609985, 10.412941455841064, 10.494908809661865]